We've gotten some questions about how to tell if a given patch is valid or not. The quickest way is to use the unidiff library installed as a `kprize` dependency.

You can also write the patch to a file and call `patch_dry_run_succeeds` for a more robust check. The gateway applies both checks.

In [1]:
! pip install -q /kaggle/input/konwinski-prize/kprize_setup/kprize-1.1.0-py3-none-any.whl --no-index --find-links /kaggle/input/konwinski-prize/kprize_setup/kprize_setup/pip_packages/kprize

In [2]:
import subprocess
from pathlib import Path

import unidiff


def is_valid_patch_format(patch: str) -> bool:
    """
    A quick check to confirm if a patch could be valid.
    """
    if not(isinstance(patch, str)):
        return False
    try:
        patch_set = unidiff.PatchSet(patch)
        if len(patch_set) == 0:
            return False
    except Exception:
        return False
    return True


def patch_dry_run_succeeds(patch_path: Path, repo_path: Path, timeout: int=60) -> bool:
    """
    A robust check if the patch will proceed without any errors.
    Should be run after `is_valid_patch_format()`: the patch
    command can hang if the inputs are sufficiently invalid.

    Args:
        patch_path: Path to a file containing the patch.
        repo_path: Path to the directory to be patched.
        timeout: Number of seconds before the dry run will be cancelled.
    """
    cmd = f"patch --quiet --dry-run -p1 -i {patch_path} -d {repo_path}"
    try:
        subprocess.run(cmd, shell=True, check=True, timeout=timeout)
        return True
    except subprocess.CalledProcessError:
        return False

In [3]:
# This should demo patch should fail.
is_valid_patch_format('Hullo world')

False